In [1]:
import json
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt
import pickle
import math
import os

In [2]:
print("Current working directory: {0}".format(os.getcwd()))

od_data = np.load('od_matrix.npy')
duration_data = np.load('duration_matrix.npy')
distance_data = np.load('distance_matrix.npy')

print("od_data.shape: ", od_data.shape)
print("duration_data.shape: ", duration_data.shape)
print("distance_data.shape: ", distance_data.shape)

demand_per_time = np.zeros(od_data.shape[2])
for t in range(od_data.shape[2]):
    demand_per_time[t] = od_data[:,:,t].sum()
print("demand per time:", demand_per_time)
low_demand_times = demand_per_time < demand_per_time.max()*0.7

time_granularity = 0.25 # in h

od_data = od_data[:, :, 8:-4] # in requests
duration_data = duration_data[:, :, 8:-4] # in sec
distance_data = distance_data[:, :, 8:-4] # in miles

print(np.percentile(od_data.flatten(), 20))
print(np.percentile(duration_data.flatten(), 20))
print(np.percentile(distance_data.flatten(), 20))

# sum across axis 2
sum_across_od = od_data.sum(axis=(0, 1))
print(sum_across_od.shape)
print("peak demand:", max(sum_across_od.flatten()))

print("20 of peak demand", 0.2*max(sum_across_od.flatten()))
print("cars per station", 0.2*0.2*max(sum_across_od.flatten())/(od_data.shape[0]))

# set remaing nans to avg
avg_duration = int(np.nanmean(duration_data))
duration_data[np.isnan(duration_data)] = avg_duration
avg_distance = int(np.nanmean(distance_data))
distance_data[np.isnan(distance_data)] = avg_distance
delta_c = 2 # energy step [kWh] 0.75

chevy_bolt_capacity = 65 # in kWh
chevy_bolt_usable_capacity = chevy_bolt_capacity * 0.6 # never go below 20% or above 80% of charge
charger_capacity = 50.0 # assuming 50. kW Chargers
assert (delta_c/charger_capacity)/time_granularity < 1
charge_levels_per_charge_step = math.floor((charger_capacity*time_granularity)/delta_c)
# print(charge_levels_per_charge_step)
chevy_bolt_range = 230 # range in mi for mild city trips according to https://media.chevrolet.com/media/us/en/chevrolet/2022-bolt-euv-bolt-ev.detail.html/content/Pages/news/us/en/2021/feb/0214-boltev-bolteuv-specifications.html
chevy_bolt_usable_range = chevy_bolt_range*0.6 # never go below 20% or above 80% of charge and assume 10% less efficient because of range https://cleantechnica.com/2017/10/13/autonomous-cars-shorter-range-due-high-power-consumption-computers/
chevy_bolt_kwh_per_mi = (chevy_bolt_usable_capacity/chevy_bolt_usable_range)/0.7 # we lose 30% efficiency because of AVs (https://medium.com/@teraki/energy-consumption-required-by-edge-computing-reduces-a-autonomous-cars-mileage-with-up-to-30-46b6764ea1b7)
# print((distance_data * chevy_bolt_kwh_per_mi)/delta_c)
# print(chevy_bolt_kwh_per_mi)
energy_distance = np.ceil(((distance_data * chevy_bolt_kwh_per_mi)/delta_c).max(axis=2))
energy_distance[energy_distance==0] = 1 # we should always use energy to satisfy a trip
print("energy_distance: ", energy_distance)
np.save('energy_distance.npy', energy_distance)
# print(np.sum(energy_distance==1.))
# print(energy_distance.max())
duration_data = np.round(duration_data/(3600*time_granularity)) # convert travel time from sec to h
duration_data[duration_data==0] = 1. # it should always take time to satisfy a trip
data_timespan = od_data.shape[2]
episode_length = int(data_timespan/time_granularity)
# https://www.pge.com/tariffs/assets/pdf/tariffbook/ELEC_SCHEDS_BEV.pdf for prices of energy

p_energy = np.ones(int(24/time_granularity))*0.16872 # in $/kWh
p_energy[int(16/time_granularity):int(21/time_granularity)] = 0.38195 # peak prices
p_energy[int(9/time_granularity):int(14/time_granularity)] = 0.14545 # super off peak prices
p_energy *= delta_c # in $/ charge level

p_energy = p_energy[int(8 * (1/time_granularity)):int(20 * (1/time_granularity))]

print(p_energy)

p_travel = 0.0770 # [$ / mi] https://newsroom.aaa.com/wp-content/uploads/2021/08/2021-YDC-Brochure-Live.pdf 
avg_miles_per_h_driving = 9.6 # in m/h from 190x190x24
operational_cost_per_timestep = avg_miles_per_h_driving * time_granularity * p_travel
print("operational_cost_per_timestep: ", operational_cost_per_timestep)
# fleet_size = 116616*0.088*2.5 # got number from Justin: 116616

peak_demand = max(sum_across_od.flatten())
fleet_size = peak_demand * 0.2

number_chargelevels = int(chevy_bolt_usable_capacity/delta_c)
number_spatial_nodes = od_data.shape[0]
charge_locations = np.ones(number_spatial_nodes,dtype=bool).tolist()
# cars_per_station_capacity = (np.ceil(np.ones(number_spatial_nodes)*fleet_size*0.05/number_spatial_nodes)).tolist()
cars_per_station_capacity = (np.ceil(np.ones(number_spatial_nodes)*fleet_size*0.2/number_spatial_nodes)).tolist()

print("cars_per_station_capacity: ", cars_per_station_capacity)
print("charge_levels_per_charge_step: ", charge_levels_per_charge_step)
print("number_chargelevels: ", number_chargelevels)

Current working directory: /Users/aaryansinghal/Documents/Stanford/ASL/private-RL-EAMOD-Personal/gnn-rl-for-eamod-main-SAC/data/NY/15
od_data.shape:  (15, 15, 24)
duration_data.shape:  (15, 15, 24)
distance_data.shape:  (15, 15, 24)
demand per time: [ 4381.  2699.  1602.  1386.  1675.  2823.  5613.  9031. 11047.  8983.
  7577.  6962.  7125.  7315.  7603.  7891.  8288.  9835.  9657.  9034.
  8728.  8666.  8394.  7641.]
2.0
842.7200000000005
3.180000000000007
(12,)
peak demand: 11047.0
20 of peak demand 2209.4
cars per station 29.458666666666673
energy_distance:  [[1. 1. 2. 2. 1. 1. 2. 3. 3. 4. 3. 3. 4. 2. 1.]
 [1. 1. 1. 2. 1. 1. 1. 2. 2. 3. 2. 3. 3. 1. 1.]
 [2. 1. 1. 1. 1. 2. 1. 2. 2. 3. 2. 3. 3. 1. 1.]
 [1. 2. 1. 1. 1. 2. 3. 3. 3. 4. 3. 3. 3. 2. 2.]
 [1. 1. 1. 1. 1. 1. 2. 3. 3. 4. 2. 3. 3. 1. 1.]
 [1. 1. 2. 2. 1. 1. 1. 2. 2. 3. 2. 3. 4. 1. 1.]
 [2. 1. 1. 3. 2. 1. 1. 1. 1. 2. 2. 2. 3. 1. 1.]
 [3. 2. 2. 3. 3. 2. 1. 1. 1. 2. 1. 2. 3. 2. 2.]
 [3. 2. 2. 3. 3. 2. 1. 1. 1. 2. 1. 2. 2. 2. 2.]


In [3]:
new_tripAttr = []
new_reb_time = []
new_total_acc = []
temp_demand = 0
for origin in range(duration_data.shape[0]):
    for destination in range(duration_data.shape[1]):
        for ts in range(episode_length):
            attr = defaultdict()
            attr['time_stamp'] = ts
            attr['origin'] = origin
            attr['destination'] = destination
            attr['demand'] = round(od_data[origin,destination,int(ts*time_granularity)]*time_granularity) # create equal distributed demand over granular time 
            temp_demand += attr['demand']
            attr['price'] = distance_data[origin,destination,int(ts*time_granularity)]*0.91 + duration_data[origin,destination,int(ts*time_granularity)]*time_granularity * 0.39 + 2.20 + 2.70 # in $
            new_tripAttr.append(attr)

            reb = defaultdict()
            reb['time_stamp'] = ts
            reb['origin'] = origin
            reb['destination'] = destination
            reb['reb_time'] = int(duration_data[origin,destination,int(ts*time_granularity)])
            new_reb_time.append(reb)
print(temp_demand)
# should be able to change 24 to 1, we don't change the number of vehicles across the day
for hour in range(24):
    acc = defaultdict()
    acc['hour'] = hour
    acc['acc'] = math.ceil(fleet_size)
    new_total_acc.append(acc)
new_data = defaultdict()
new_data['demand'] = new_tripAttr
new_data['rebTime'] = new_reb_time
new_data['totalAcc'] = new_total_acc
new_data['chargelevels'] = number_chargelevels
new_data['spatialNodes'] = number_spatial_nodes
new_data['chargeLevelsPerChargeStep'] = charge_levels_per_charge_step
new_data['episodeLength'] = episode_length
new_data['energy_prices'] = p_energy.tolist()
new_data['chargeLocations'] = charge_locations
new_data['carsPerStationCapacity'] = cars_per_station_capacity
new_data['timeGranularity'] = time_granularity
new_data['operationalCostPerTimestep'] = operational_cost_per_timestep
new_data['peakHours'] = [32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]

print(charge_levels_per_charge_step)
print(episode_length)
with open(f'NYC_{number_spatial_nodes}.json', 'w') as f:
    json.dump(new_data, f)

100892
6
48
